**Load Tables**

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pickle

In [ ]:
with open('/content/gdrive/MyDrive/Work/Aarete (Fall 2021)/Molina Letters/tables_for_letter.pkl', 'rb') as f:
  tables_for_letter = pickle.load(f)

for table in tables_for_letter:
  try:
    table.iloc[19, 1] = int(table.iloc[19, 1]) # make zipcode int
  except ValueError:
    pass

  for i in range(23, 37): # make datetime to readable format 
    try:
      table.iloc[i, 1] = table.iloc[i, 1].strftime('%I:%M %p')
    except AttributeError:
      pass

len(tables_for_letter)

429

In [ ]:
for table in tables_for_letter:
  # ensure all digits are integers
  try:
    table.iloc[20, 1] = int(table.iloc[20, 1])
  except ValueError:
    pass

**Merge Fields**

In [ ]:
!pip install docx-mailmerge
from mailmerge import MailMerge
import datetime

In [ ]:
date = str(datetime.date.today())

for table in tables_for_letter:
  letter = MailMerge('/content/gdrive/MyDrive/Work/Aarete (Fall 2021)/Molina Letters/Letter Template.docx')
  first_name = table[table['Provider Data Fields'] == 'First_Name'].iloc[:,1].values[0]
  last_name = table[table['Provider Data Fields'] == 'Last_Name/ Facility_Name'].iloc[:,1].values[0]
  if first_name:
    name = f'{first_name} {last_name}'
  else:
    name = last_name
  npi = str(table.iloc[0, 1])
  address = table[table['Provider Data Fields'] == 'Address'].iloc[:,1].values[0]
  csz = f'{table[table["Provider Data Fields"] == "City"].iloc[:,1].values[0]}, {table[table["Provider Data Fields"] == "State"].iloc[:,1].values[0]} {table[table["Provider Data Fields"] == "ZIP"].iloc[:,1].values[0]}'
  letter.merge(Address=address, Name=name, Date=date, CSZ=csz)
  letter.write(f'/content/gdrive/MyDrive/Work/Aarete (Fall 2021)/Molina Letters/Letters/{name}_{npi}_({address}, {csz}).docx')


**Add Table**

In [ ]:
!pip install python-docx
import docx
import pandas as pd
from glob import glob
import numpy as np

     |████████████████████████████████| 5.6 MB 5.4 MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184508 sha256=5349b4c491cd50c013496d9f0608b37001310bde75ad162efd06f7127ba87f79
  Stored in directory: /root/.cache/pip/wheels/f6/6f/b9/d798122a8b55b74ad30b5f52b01482169b445fbb84a11797a6
Successfully built python-docx


In [ ]:
letters = glob('/content/gdrive/MyDrive/Work/Aarete (Fall 2021)/Molina Letters/Letters/*.docx')
for table in tables_for_letter:
  first_name = table[table['Provider Data Fields'] == 'First_Name'].iloc[:,1].values[0]
  last_name = table[table['Provider Data Fields'] == 'Last_Name/ Facility_Name'].iloc[:,1].values[0]
  if first_name:
    name = f'{first_name} {last_name}'
  else:
    name = last_name
  npi = str(table.iloc[0, 1])
  
  address = table[table['Provider Data Fields'] == 'Address'].iloc[:,1].values[0]
  csz = f'{table[table["Provider Data Fields"] == "City"].iloc[:,1].values[0]}, {table[table["Provider Data Fields"] == "State"].iloc[:,1].values[0]} {table[table["Provider Data Fields"] == "ZIP"].iloc[:,1].values[0]}'
  filename = f'{name}_{npi}_({address}, {csz})'

  try:
    letter = [letter for letter in letters if filename in letter][0]
  except IndexError:
    continue
  doc = docx.Document(letter)
  doc.add_page_break()

  # add a table to the end and create a reference variable
  # extra row is so we can add the header row
  t = doc.add_table(table.shape[0]+1, table.shape[1])
  t.style = 'Table Grid'

  # add the header rows.
  for j in range(table.shape[-1]):
      t.cell(0,j).text = table.columns[j]

  # add the rest of the data frame
  for i in range(table.shape[0]):
      for j in range(table.shape[-1]):
          t.cell(i+1,j).text = str(table.values[i,j])

  # save the doc
  doc.save(letter) 
